# Core 4.2 Batch Pipelines - Local Pipelines

In this section, we will execute the pipeline we created in the last section using local execution. This is a good option for testing the pipeline locally in your Jupyter/IDE environment before running on Kubeflow pipelines.

Not that local pipelines are not as fully featured as Kubeflow pipelines and do not support branches/conditions. That being said, they are execellent for prototyping simple pipelines like the one we created.

---

### References

Much of the following content is derived from the official documenation:
- [Project workflows and automation](https://docs.mlrun.org/en/latest/projects/workflows.html)

### Example Overview

In this example, we will execute the pipeline we created in the first section via the local pipeline engine. To run the pipeline, we will use the MLRun Project object. The project knows where the underlying files for the pipeline and functions are, making it very simple to run via the Python SDK or CLI.

---

### Setup

In [1]:
import mlrun

project = mlrun.get_or_create_project("iguazio-academy", context="./")

> 2022-06-23 21:01:47,332 [info] loaded project iguazio-academy from MLRun DB


### List Pipelines in Project

Recall that we saved our pipeline in the project under the name `train`

In [2]:
project.workflows

[{'name': 'train', 'path': 'pipelines/training_pipeline.py', 'engine': None}]

### Run Pipeline via Python SDK

To run locally, we need to add the `local` flag - that's it

In [5]:
run_id = project.run(
    name="train",
    arguments={'label_column': 'target', 'test_size' : 0.2}, # Arguments in Key Value format
    sync=True, # Get latest versions of all MLRun functions before running,
    watch=True, # Wait for pipeline to execute before continuing (prints out graph)
    local=True
)

> 2022-06-23 21:05:04,361 [info] starting run get-data-prep_data uid=45d779e3141d4414b489654aa8b45b17 DB=http://mlrun-api:8080
> 2022-06-23 21:05:04,536 [info] saving iris dataframe to v3io:///projects/iguazio-academy/artifacts/9ab247cb82414b6a928fcb31182dc943


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
iguazio-academy,...a8b45b17,0,Jun 23 21:05:04,completed,get-data-prep_data,workflow=9ab247cb82414b6a928fcb31182dc943v3io_user=nickkind=owner=nickhost=jupyter-nick-85b8d8d8df-tgstz,,label_column=target,num_rows=150,iris_dataset


> 2022-06-23 21:05:04,731 [info] run executed, status=completed
> 2022-06-23 21:05:04,733 [info] starting run train-model-train_model uid=42e4293e72464dae980718ecf8b36495 DB=http://mlrun-api:8080


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
iguazio-academy,...f8b36495,0,Jun 23 21:05:04,completed,train-model-train_model,workflow=9ab247cb82414b6a928fcb31182dc943v3io_user=nickkind=owner=nickhost=jupyter-nick-85b8d8d8df-tgstz,dataset,label_column=targettest_size=0.2,accuracy=1.0f1_score=1.0precision_score=1.0recall_score=1.0auc-micro=1.0auc-macro=1.0auc-weighted=1.0,feature-importancetest_setconfusion-matrixroc-curvesmodel


> 2022-06-23 21:05:06,299 [info] run executed, status=completed
> 2022-06-23 21:05:06,300 [info] Starting remote function deploy
2022-06-23 21:05:06  (info) Deploying function
2022-06-23 21:05:06  (info) Building
2022-06-23 21:05:06  (info) Staging files and preparing base images
2022-06-23 21:05:06  (info) Building processor image
2022-06-23 21:05:41  (info) Build complete
2022-06-23 21:05:51  (info) Function deploy complete
> 2022-06-23 21:05:52,732 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-iguazio-academy-v2-model-server.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['iguazio-academy-v2-model-server-iguazio-academy.default-tenant.app.uss-sales-341.iguazio-cd1.com/']}


uid,start,state,name,parameters,results
...a8b45b17,Jun 23 21:05:04,completed,get-data-prep_data,label_column=target,num_rows=150
...f8b36495,Jun 23 21:05:04,completed,train-model-train_model,label_column=targettest_size=0.2,accuracy=1.0f1_score=1.0precision_score=1.0recall_score=1.0auc-micro=1.0auc-macro=1.0auc-weighted=1.0


9ab247cb82414b6a928fcb31182dc943

### Run Pipeline via CLI

Like the Python SDK, the CLI has many options for running projects. The CLI is often used in CI/CD pipelines rather than the Python SDK, but that is user preference. Like before, we need to add the `local` flag

Run `mlrun project --help` for more info. To execute our pipeline, run the following:

In [7]:
!mlrun project ./ --run train --arguments label_column=target --arguments test_size=0.2 --local

Loading project iguazio-academy into ./:

kind: project
metadata:
  name: iguazio-academy
  created: '2022-06-23T19:20:05.968000+00:00'
spec:
  functions:
  - url: functions/get_data.py
    name: get-data
    kind: job
    image: mlrun/mlrun
  - url: functions/train.py
    name: train-model
    kind: job
    image: mlrun/mlrun
  - url: hub://v2_model_server
    name: deploy-model
  workflows:
  - name: train
    path: pipelines/training_pipeline.py
    engine: null
  artifacts: []
  source: ''
  desired_state: online
  owner: xingsheng
status:
  state: online

running workflow train file: None
Pipeline started in project iguazio-academy id=4d3c6769bbe6420789a54ab5857b9cc3, check progress in https://dashboard.default-tenant.app.uss-sales-341.iguazio-cd1.com/mlprojects/iguazio-academy/jobs
> 2022-06-23 21:08:39,889 [info] starting run get-data-prep_data uid=f7a693c8f50c4c81aee25b1eaacb7200 DB=http://mlrun-api:8080
> 2022-06-23 21:08:40,841 [info] saving iris dataframe to v3io:///projects

---